In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/nika/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')
df.head()

/tmp/ipykernel_58095/3954121827.py:4: DtypeWarning: Columns (24,25,26,35,43,53,59,63,74,75,85,89,108,110,116,120,122,124,136,141,147,150,151,162,163,177,178,180,195,214,221,226,234,241,246,259,266,270,271,273,278,283,287,290) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/nika/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')


,Unnamed: 0.1,Unnamed: 0,Hot air,Volumetric anti -first alarm,Harman Kardon Sound System,Track change alert,Carbonaceramic brake disc,Arconditional Zones 2,Reclinarinar rear seat,Maximum rotation,...,Folding rear seat,Free wheel,Shaker Pro Audio Sound System,Maximum speed electric mode,Electronic noise cancellation,Steering wheel adjustment in depth,Facial recognition camera,Carcode,Perimeter anti theft alarm,Track centralization assistant
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN


## Too many features with NaN's, remove columns which have more then 15000 Nan'n in column

In [300]:
filtered_columns = df.columns[df.isna().sum() < 15000]
filtered_df = df[filtered_columns]
filtered_df

,Unnamed: 0.1,Hot air,Acceleration,Length,Maximum torque,Assistance,Width,Rev counter,Compression ratio,Assisted direction,...,USB connection,Gear speed transmissions,Corrected frontal area,Urban,Platform,Year,Radio,Folding rear seat,Carcode,Perimeter anti theft alarm
0,0,NaN,"0100 km/h 3,8 s",4580 mm,"66,3 kgfm",Hydraulics,2045 mm,NaN,"10,71",NaN,...,NaN,6.0,"0,650 m^2","2,4 km/l",NaN,2002.0,NaN,NaN,1,NaN
1,1,NaN,"0100 km/h 15,2 s",3644 mm,"8,5 kgfm",Unattended,1548 mm,NaN,"9,51",NaN,...,NaN,5.0,"0,684 m^2",10 km/l,NaN,2002.0,NaN,NaN,2,NaN
2,2,NaN,"0100 km/h 10,6 s",4177 mm,"17,3 kgfm",Hydraulics,1709 mm,NaN,"9,81",NaN,...,NaN,5.0,"0,624 m^2","7,2 km/l",Tbody,2002.0,NaN,NaN,3,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN
4,4,NaN,"0100 km/h 8,5 s",4149 mm,"21,4 kgfm",Hydraulics,1735 mm,NaN,"9,51",NaN,...,NaN,5.0,"0,651 m^2","6,8 km/l",PQ34,2002.0,NaN,NaN,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23899,23899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23902,NaN
23900,23900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23903,NaN
23901,23901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23904,NaN
23902,23902,Standard equipment,"0100 km/h 14,9 s",5413 mm,"34,7 kgfm",Hydraulics,2050 mm,Standard equipment,161,Standard equipment,...,Standard equipment,6.0,NaN,10 km/l,NaN,2023.0,Optional equipment,NaN,23905,Optional equipment


## Drop some rows with a lot of Nan's

In [386]:
test_filtered_df =filtered_df.dropna(subset='Acceleration')
test_filtered_df

,Unnamed: 0.1,Hot air,Acceleration,Length,Maximum torque,Assistance,Width,Rev counter,Compression ratio,Assisted direction,...,USB connection,Gear speed transmissions,Corrected frontal area,Urban,Platform,Year,Radio,Folding rear seat,Carcode,Perimeter anti theft alarm
0,0,NaN,"0100 km/h 3,8 s",4580 mm,"66,3 kgfm",Hydraulics,2045 mm,NaN,"10,71",NaN,...,NaN,6.0,"0,650 m^2","2,4 km/l",NaN,2002.0,NaN,NaN,1,NaN
1,1,NaN,"0100 km/h 15,2 s",3644 mm,"8,5 kgfm",Unattended,1548 mm,NaN,"9,51",NaN,...,NaN,5.0,"0,684 m^2",10 km/l,NaN,2002.0,NaN,NaN,2,NaN
2,2,NaN,"0100 km/h 10,6 s",4177 mm,"17,3 kgfm",Hydraulics,1709 mm,NaN,"9,81",NaN,...,NaN,5.0,"0,624 m^2","7,2 km/l",Tbody,2002.0,NaN,NaN,3,NaN
4,4,NaN,"0100 km/h 8,5 s",4149 mm,"21,4 kgfm",Hydraulics,1735 mm,NaN,"9,51",NaN,...,NaN,5.0,"0,651 m^2","6,8 km/l",PQ34,2002.0,NaN,NaN,5,NaN
5,5,NaN,"0100 km/h 9,5 s",4187 mm,"16,7 kgfm",Hydraulics,1741 mm,NaN,"10,31",NaN,...,NaN,5.0,NaN,9 km/l,NaN,2002.0,NaN,NaN,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23881,23881,Standard equipment,"0100 km/h 10,7 s",4015 mm,"17,5 kgfm A",Electrical,1720 mm,Standard equipment,"10,51",Standard equipment,...,Standard equipment,6.0,NaN,"8,3 km/l A",PB,2024.0,Standard equipment,Standard equipment,23884,Standard equipment
23882,23882,Standard equipment,"0100 km/h 10,7 s",4015 mm,"17,5 kgfm A",Electrical,1720 mm,Standard equipment,"10,51",Standard equipment,...,Standard equipment,6.0,NaN,"8,3 km/l A",PB,2024.0,Standard equipment,Standard equipment,23885,Standard equipment
23890,23890,Standard equipment,"0100 km/h 10,7 s",4015 mm,"17,5 kgfm A",Electrical,1720 mm,Standard equipment,"10,51",Standard equipment,...,Standard equipment,6.0,NaN,9 km/l A,PB,2024.0,Standard equipment,Standard equipment,23893,Standard equipment
23902,23902,Standard equipment,"0100 km/h 14,9 s",5413 mm,"34,7 kgfm",Hydraulics,2050 mm,Standard equipment,161,Standard equipment,...,Standard equipment,6.0,NaN,10 km/l,NaN,2023.0,Optional equipment,NaN,23905,Optional equipment


## Check how much % is NaN in every column and check Dtype

In [387]:
nan_percentage_per_column = test_filtered_df.isnull().mean() * 100

# Create a new DataFrame with column names and NaN percentages and Dtypes
nan_info_df = pd.DataFrame({
    'Column Name': nan_percentage_per_column.index,
    'NaN Percentage': nan_percentage_per_column.values,
    'Data Type': test_filtered_df.dtypes
})

# Display the new DataFrame
nan_info_df[0:50] # Cheking 50 by 50 to see whole list at a time.

,Column Name,NaN Percentage,Data Type
Unnamed: 0.1,Unnamed: 0.1,0.000000,int64
Hot air,Hot air,46.233340,object
Acceleration,Acceleration,0.000000,object
Length,Length,0.000000,object
Maximum torque,Maximum torque,0.709871,object
Assistance,Assistance,0.053120,object
Width,Width,0.000000,object
Rev counter,Rev counter,47.711030,object
Compression ratio,Compression ratio,0.922349,object
Assisted direction,Assisted direction,46.440989,object


## After analysing all features, decided to remove some of them.
Neither they don't give information, or they are hard to preprocess.


In [388]:
column_to_remove = ['Unnamed: 0.1','Assistance','Carcode','Aspiration', 'Rear tires', 'Spare tire', 'Front tires',
                    'Urban autonomy', 'Generation', 'Front suspension', 'Rear suspension', 'Coupling', 'Valve command',
                    'Places', 'Disposition', 'Cylinders', 'Elastic element', 'Ipva R', 'Frontal area A', 'Engine code',
                    'Traction', 'Installation', 'Road autonomy', 'Engine power supply', 'Engine control activation',
                    'Gear change code', 'Corrected frontal area', 'Platform']

test_filtered_df['Hot air'].unique() # check feature

array([nan, 'Standard equipment', 'Optional equipment'], dtype=object)

In [389]:
new_filtered_df = test_filtered_df.drop(columns=column_to_remove)
new_filtered_df.head(3)

,Hot air,Acceleration,Length,Maximum torque,Width,Rev counter,Compression ratio,Assisted direction,CNW Ranking,ABS brakes,...,Car size,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year,Radio,Folding rear seat,Perimeter anti theft alarm
0,NaN,"0100 km/h 3,8 s",4580 mm,"66,3 kgfm",2045 mm,NaN,"10,71",NaN,20549.0,NaN,...,Grande,516 cm^3,50 litros,NaN,6.0,"2,4 km/l",2002.0,NaN,NaN,NaN
1,NaN,"0100 km/h 15,2 s",3644 mm,"8,5 kgfm",1548 mm,NaN,"9,51",NaN,14268.0,NaN,...,Compact and portable,250 cm^3,290 litros,NaN,5.0,10 km/l,2002.0,NaN,NaN,NaN
2,NaN,"0100 km/h 10,6 s",4177 mm,"17,3 kgfm",1709 mm,NaN,"9,81",NaN,12266.0,NaN,...,Medium,500 cm^3,370 litros,NaN,5.0,"7,2 km/l",2002.0,NaN,NaN,NaN


# Preprocessing Features

## Preprocessing All features with [Nan, standart equipment, optinal equipment]
saved in: columns_to_transform ---------
Nan = 0 ----------------
standart equipment, optinal equipment = 1

In [390]:
preprocessing_df = new_filtered_df.copy()

columns_to_transform = ['Hot air', 'Rev counter', 'Assisted direction', 'ABS brakes', 'Rear window', 'Central locking of the doors',
                        'Headrest for all occupants', 'Electric rearview mirror adjustment', 'Air conditioning',
                        'Bluetooth connection', 'Frontal Airbags', 'Steering wheel adjustment height',
                        'Electric front window control', 'Multifunctional steering wheel', 'Driver s seat with height adjustment',
                        'On board computer', 'Light in the trunk', 'Alloy wheels', 'USB connection', 'Radio',
                        'Folding rear seat', 'Perimeter anti theft alarm', 'Cooling liquid thermometer']

# Replace NaN with 0 and other values with 1
preprocessing_df[columns_to_transform] = preprocessing_df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


/tmp/ipykernel_58095/589099134.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  preprocessing_df[columns_to_transform] = preprocessing_df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


## Preprocessing [Acceleration]

In [391]:
# Preprocess Acceleration (0100 km/h 3,8 s = 3.8)
preprocessing_df['Acceleration'] = preprocessing_df['Acceleration'].str.extract(r'(\d+\,\d+)')

# Replace ',' with '.' and convert to numeric
preprocessing_df['Acceleration'] = preprocessing_df['Acceleration'].str.replace(',', '.').astype(float)
# Rename Column
preprocessing_df = preprocessing_df.rename(columns={'Acceleration': 'Acceleration 0100 km/h in S'})

In [395]:
preprocessing_df['Maximum torque'].unique()
#preprocessing_df['Drag coefficient'].dtype

array(['66,3 kgfm', '8,5 kgfm', '17,3 kgfm', '21,4 kgfm', '16,7 kgfm',
       '23,9 kgfm', '33,3 kgfm', '8,3 kgfm', '8,9 kgfm', '28,5 kgfm',
       '15,2 kgfm', '18,2 kgfm', '9,5 kgfm', '9,7 kgfm', '9,6 kgfm',
       '29 kgfm', '19,6 kgfm', '15 kgfm', '19,2 kgfm', '57,1 kgfm',
       '14,8 kgfm', '15,1 kgfm', '19,1 kgfm', '17,5 kgfm', '44,9 kgfm',
       '30,6 kgfm', '21,2 kgfm', '21,9 kgfm', '19,8 kgfm', '32,1 kgfm',
       '15,4 kgfm', '9,2 kgfm', '14,2 kgfm', '40,8 kgfm', '18,6 kgfm',
       '12 kgfm', '31,6 kgfm', '33,6 kgfm', '8,6 kgfm', '15,5 kgfm',
       '39 kgfm', '19,4 kgfm', '16,3 kgfm', '30,8 kgfm', '18,5 kgfm',
       '16,1 kgfm', '29,9 kgfm', '13 kgfm', '39,8 kgfm', '17,7 kgfm',
       '50,1 kgfm', '27 kgfm', '25 kgfm', '40 kgfm', '18,4 kgfm',
       '15,8 kgfm', '7,5 kgfm', '8,1 kgfm', '50 kgfm', '34,7 kgfm',
       '14,3 kgfm', '13,9 kgfm', '21 kgfm', '29,8 kgfm', '14,1 kgfm',
       '17 kgfm', '39,6 kgfm', '32 kgfm', '22,6 kgfm', '27,2 kgfm',
       '38,7 kgfm', '29,6 

## Preprocessing [All features with strings]

In [346]:
columns_to_transform_mm = ['Length', 'Width'] # add more
columns_to_transform_kgfm = ['Maximum torque']

# Remove strings from each value in the column
preprocessing_df[columns_to_transform_mm] = preprocessing_df[columns_to_transform_mm].str.replace(' mm', '')
preprocessing_df['Maximum torque'] = preprocessing_df['Maximum torque'].str.replace(' kgfm', '')

# Convert columns to numeric type
preprocessing_df[columns_to_transform_mm] = pd.to_numeric(preprocessing_df[columns_to_transform_mm])
preprocessing_df['Maximum torque'] = pd.to_numeric(preprocessing_df['Maximum torque'])

# Rename columns
preprocessing_df = preprocessing_df.rename(columns={'Length': 'Length mm',
                                                    'Width': 'Width mm',
                                                    'Maximum torque': 'Maximum torque kgfm'})

SyntaxError: invalid syntax (3517212674.py, line 14)

In [ ]:
columns_to_transform_with_string_at_the_end = ['Weight/Torque', 'Length', 'Maximum torque', 'Guarantee']

In [394]:
def extract_numerical_value(value):
    try:
        numerical_value = value.split()[0].replace(',', '.')
        return float(numerical_value)
    except (ValueError, IndexError):
        return np.nan

# test = preprocessing_df[['Weight/Torque', 'Length', 'Maximum torque', 'Guarantee']]
for column in preprocessing_df[['Weight/Torque', 'Length', 'Maximum torque', 'Guarantee']]:
    test[column] = test[column].apply(extract_numerical_value)
test

AttributeError: 'float' object has no attribute 'split'

In [379]:
def convert_to_float_array(data):
    # Replace commas with dots and convert to floats
    data_float = np.array.astype(float)
    return data_float

test_1 = preprocessing_df['Drag coefficient']
test_1

0        0,33
1        0,36
2         0,3
4        0,31
5         NaN
         ... 
23881     NaN
23882     NaN
23890     NaN
23902     NaN
23903     NaN
Name: Drag coefficient, Length: 20708, dtype: object

In [57]:
# filtered_df.to_csv("df_main_features.csv")

In [320]:
preprocessing_df.dtypes

Hot air                          int64
Acceleration 0100 km/h in S    float64
Length                           int64
Maximum torque                  object
Width                           object
                                ...   
Urban                           object
Year                           float64
Radio                            int64
Folding rear seat                int64
Perimeter anti theft alarm       int64
Length: 71, dtype: object

In [309]:
test_list = []

for i in preprocessing_df.columns:
    if i not in columns_to_transform:
        test_list.append(i)

print(test_list)


['Acceleration 0100 km/h in S', 'Length', 'Maximum torque', 'Width', 'Compression ratio', 'CNW Ranking', 'Tuching', 'Weight/Torque', 'Guarantee', 'Drag coefficient', 'Height', 'Valves per cylinder', 'Specific torque', 'Minimum height from the ground', 'Propulsion', 'Piston course', 'Car gearbox', 'Weight', 'Price R', 'Front gauge', 'Fuel', 'Weight/power', 'Displacement', 'Max power regime.', 'Turns diameter', 'Cylinder diameter', 'Rear gauge', 'Length between the axis', 'Maximum speed', 'Fuel tank', 'Devaluation', 'Road consumption', 'Max torque regime', 'Car payload', 'Specific power', 'CNW Index', 'Reader score', 'Sidewall height', 'Provenance', 'Doors', 'Maximum power', 'Settings', 'Car size', 'Unit displacement', 'Trunk', 'Gear speed transmissions', 'Urban', 'Year']


In [384]:
import sys
print(sys.path)

['/home/nika/code/marcnaweb/car_recommendation_engine/notebooks', '/home/nika/code/elnika1/04-Decision-Science/01-Project-Setup/data-context-and-setup', '/home/nika/code/marcnaweb/car_recommendation_engine/notebooks', '/home/nika/.pyenv/versions/3.10.6/lib/python310.zip', '/home/nika/.pyenv/versions/3.10.6/lib/python3.10', '/home/nika/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload', '', '/home/nika/.pyenv/versions/3.10.6/envs/car_recommendation_engine/lib/python3.10/site-packages']


In [398]:
miau = {'Weight/Torque': 'kg',
 'Weight': 'kg',
 'Weight/power': 'kg',
 'Max power regime.': 'rpm',
 'Cylinder diameter': 'mm',
 'Fuel tank': 'litros',
 'Specific power': 'cv',
 'Maximum power': 'cv',
 'Length': 'mm',
 'Maximum torque': 'kgfm',
 'Width': 'mm',
 'Height': 'mm',
 'Specific torque': 'kgfm',
 'Minimum height from the ground': 'mm',
 'Piston course': 'mm',
 'Front gauge': 'mm',
 'Displacement': 'cm',
 'Turns diameter': 'm',
 'Rear gauge': 'mm',
 'Length between the axis': 'mm',
 'Maximum speed': 'km',
 'Road consumption': 'km',
 'Max torque regime': 'rpm',
 'Car payload': 'kg',
 'Sidewall height': 'mm',
 'Unit displacement': 'cm',
 'Trunk': 'litros',
 'Urban': 'km'}

keysList = list(miau.keys())
print(keysList)

['Weight/Torque', 'Weight', 'Weight/power', 'Max power regime.', 'Cylinder diameter', 'Fuel tank', 'Specific power', 'Maximum power', 'Length', 'Maximum torque', 'Width', 'Height', 'Specific torque', 'Minimum height from the ground', 'Piston course', 'Front gauge', 'Displacement', 'Turns diameter', 'Rear gauge', 'Length between the axis', 'Maximum speed', 'Road consumption', 'Max torque regime', 'Car payload', 'Sidewall height', 'Unit displacement', 'Trunk', 'Urban']
